In [80]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
games = pd.read_parquet('../Datasets/gamesoh.parquet')
reviews = pd.read_parquet('../Datasets/reviews.parquet')
items = pd.read_parquet('../Datasets/itemso.parquet')

generos = list(games.drop(columns=['item_id']).columns)
gnames = pd.read_parquet('../Datasets/games.parquet')
fnames = pd.read_parquet('../Datasets/fnames.parquet')
unames = pd.read_parquet('../Datasets/unames.parquet')

lid=list(set(gnames['item_id']))
lfid=list(set(fnames['item_id']))
lanio=list(set(reviews['posted'].to_list()))

In [79]:
items.head()

,item_id,item_name,playtime_2weeks,playtime_forever,user_id
0,10,Counter-Strike,0.0,6.0,615
1,20,Team Fortress Classic,0.0,0.0,615
2,30,Day of Defeat,0.0,7.0,615
3,40,Deathmatch Classic,0.0,0.0,615
4,50,Half-Life: Opposing Force,0.0,0.0,615


In [82]:
# Obtener el uso de la memoria por columna
m1 = games.memory_usage(deep=True)
m2 = reviews.memory_usage(deep=True)
m3 = items.memory_usage(deep=True)
m4 = gnames.memory_usage(deep=True)
m5 = fnames.memory_usage(deep=True)
m6 = unames.memory_usage(deep=True)
# Sumar el uso total de la memoria
t1 = m1.sum() / (1024 ** 2)  # Convertir a megabytes
t2 = m2.sum() / (1024 ** 2)  # Convertir a megabytes
t3 = m3.sum() / (1024 ** 2)  # Convertir a megabytes
t4 = m4.sum() / (1024 ** 2)  # Convertir a megabytes
t5 = m5.sum() / (1024 ** 2)  # Convertir a megabytes
t6 = m6.sum() / (1024 ** 2)  # Convertir a megabytes

print(f"Uso total de la memoria: {t1:.2f} MB")
print(f"Uso total de la memoria: {t2:.2f} MB")
print(f"Uso total de la memoria: {t3:.2f} MB")
print(f"Uso total de la memoria: {t4:.2f} MB")
print(f"Uso total de la memoria: {t5:.2f} MB")
print(f"Uso total de la memoria: {t6:.2f} MB")

Uso total de la memoria: 1.85 MB
Uso total de la memoria: 6.66 MB
Uso total de la memoria: 118.97 MB
Uso total de la memoria: 3.04 MB
Uso total de la memoria: 0.04 MB
Uso total de la memoria: 6.68 MB


In [3]:
games.head()

,item_id,2D,ACTION,ADVENTURE,ANIME,ARCADE,ATMOSPHERIC,CASUAL,CLASSIC,CO-OP,...,STORY RICH,STRATEGY,SURVIVAL,TACTICAL,THIRD PERSON,TURN-BASED,UTILITIES,VIOLENT,VISUAL NOVEL,ZOMBIES
0,10,0,1,0,0,0,0,0,1,0,...,0,1,1,1,0,0,0,0,0,0
1,20,0,1,1,0,0,0,1,1,1,...,1,0,0,0,0,0,0,0,0,0
2,30,0,1,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
3,40,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,1,1,0,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0


In [31]:
games[['item_id','2D']]

,item_id,2D
0,10,0
1,20,0
2,30,0
3,40,0
4,50,0
...,...,...
29798,2028055,0
29799,2028056,0
29800,2028062,0
29801,2028103,0


In [4]:
reviews.head()

,posted,item_id,recommend,user_id,sentiment_analysis
0,2014,1250,True,0,1
1,2014,113200,True,1,2
2,2014,440,True,1,2
3,2014,730,True,2,2
4,2015,105600,True,2,2


In [5]:
items.head()

,item_id,item_name,playtime_2weeks,playtime_forever,user_id
0,10,Counter-Strike,0.0,6.0,615
1,20,Team Fortress Classic,0.0,0.0,615
2,30,Day of Defeat,0.0,7.0,615
3,40,Deathmatch Classic,0.0,0.0,615
4,50,Half-Life: Opposing Force,0.0,0.0,615


In [6]:
unames.head()

,user_id,user_name
0,0,--000--
1,1,--ace--
2,2,--ionex--
3,3,-2SV-vuLB-Kg
4,4,-Azsael-


In [7]:
gnames.head()

,title,release_date,item_id
120416,Counter-Strike,2000,10
120413,Team Fortress Classic,1999,20
120424,Day of Defeat,2003,30
120418,Deathmatch Classic,2001,40
120414,Half-Life: Opposing Force,1999,50


In [38]:
fnames.head()

,item_id,item_name
0,277950,Deadbreed®
1,226700,Infestation: Survivor Stories 2020
2,244850,Space Engineers
3,218620,PAYDAY 2
4,428190,MadOut Open City


In [10]:
gnames.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29803 entries, 120416 to 89735
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         29802 non-null  object
 1   release_date  29803 non-null  int32 
 2   item_id       29803 non-null  int32 
dtypes: int32(2), object(1)
memory usage: 698.5+ KB


In [11]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29803 entries, 0 to 29802
Data columns (total 62 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   item_id                29803 non-null  int32
 1   2D                     29803 non-null  int8 
 2   ACTION                 29803 non-null  int8 
 3   ADVENTURE              29803 non-null  int8 
 4   ANIME                  29803 non-null  int8 
 5   ARCADE                 29803 non-null  int8 
 6   ATMOSPHERIC            29803 non-null  int8 
 7   CASUAL                 29803 non-null  int8 
 8   CLASSIC                29803 non-null  int8 
 9   CO-OP                  29803 non-null  int8 
 10  COMEDY                 29803 non-null  int8 
 11  CUTE                   29803 non-null  int8 
 12  DESIGN & ILLUSTRATION  29803 non-null  int8 
 13  DIFFICULT              29803 non-null  int8 
 14  EARLY ACCESS           29803 non-null  int8 
 15  EXPLORATION            29803 non-nul

In [12]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5197727 entries, 0 to 5197726
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_2weeks   float64
 3   playtime_forever  float64
 4   user_id           int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 198.3+ MB


In [13]:
items['item_id'] = items['item_id'].astype(int)

In [14]:
items['user_id'] = items['user_id'].astype(int)

In [15]:
unames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87998 entries, 0 to 87997
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    87998 non-null  int64 
 1   user_name  87998 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [16]:
unames['user_id'] = unames['user_id'].astype(int)

In [66]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   posted              59333 non-null  int32 
 1   item_id             59305 non-null  object
 2   recommend           59305 non-null  object
 3   user_id             59333 non-null  int64 
 4   sentiment_analysis  59333 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 2.0+ MB


In [70]:
reviews.dropna(subset='item_id', inplace =True)

In [71]:
reviews['item_id'] = reviews['item_id'].astype(int)

In [19]:
def PlayTimeGenre(genero):
  genero=genero.upper()
  if genero in generos:
    lgames = games.loc[games[genero] == 1, ['item_id']]
    lhoras = items.groupby('item_id')['playtime_forever'].sum().reset_index()
    dfplay = pd.merge(lgames, lhoras, on='item_id', how='inner')
    dfplay = pd.merge(dfplay, gnames, on='item_id', how='inner')
    df_horas = dfplay.groupby('release_date')['playtime_forever'].sum().reset_index()
    df_horas_ordenado = df_horas.sort_values(by='playtime_forever', ascending=False)
    resultado=df_horas_ordenado.head(1)
    prin = "Año de lanzamiento con más horas jugadas para Género " + genero
    anio = int(resultado['release_date'].iloc[0])
    return {prin : anio}
  else:
    return {"El Genero indicado no se encuentra en la base de datos":genero}

In [20]:
PlayTimeGenre("2d")

{'Año de lanzamiento con más horas jugadas para Género 2D': 2011}

In [34]:
def UserForGenre(genero):
  genero=genero.upper()
  if genero in generos:
    dfgames = games[['item_id',genero]]
    dfgames = dfgames[games[genero]==1]
    dfuserg = pd.merge(dfgames,items,on='item_id',how='inner')
    uhoras = dfuserg.groupby('user_id')['playtime_forever'].sum().reset_index()
    uhoras = uhoras.sort_values(by='playtime_forever', ascending=False)
    uhoras = uhoras.head(1)
    uhoras = uhoras['user_id'].iloc[0]
    dfuserg = dfuserg[dfuserg['user_id']==uhoras]
    dfuserg = pd.merge(dfuserg,gnames,on='item_id',how='inner')
    dfuserg = dfuserg.groupby('release_date')['playtime_forever'].sum().reset_index()
    dfuserg = dfuserg.sort_values(by='release_date', ascending=False)
    dfuserg = dfuserg[dfuserg['playtime_forever']>0]
    dfuserg['playtime_forever'] = dfuserg['playtime_forever'].astype(int)
    dfuserg = dfuserg[['playtime_forever', 'release_date']]
    dfuserg.rename(columns={'release_date': 'Año', 'playtime_forever':'Horas'}, inplace=True)
    di = dfuserg.apply(lambda row: row.to_dict(), axis=1).tolist()
    uhoras = unames[unames['user_id']==uhoras]
    uhoras = uhoras['user_id'].iloc[0]
    prin="Usuario con más horas jugadas para Género " + genero
    return {prin:uhoras,"Horas Jugadas":di}
  else:
    return {"El Genero indicado no se encuentra en la base de datos ":genero}

In [35]:
UserForGenre('AcTiOn')

{'Usuario con más horas jugadas para Género ACTION': 16914,
 'Horas Jugadas': [{'Horas': 43327, 'Año': 2017},
  {'Horas': 30195, 'Año': 2016},
  {'Horas': 394314, 'Año': 2015},
  {'Horas': 132864, 'Año': 2014},
  {'Horas': 122158, 'Año': 2013},
  {'Horas': 384238, 'Año': 2012},
  {'Horas': 155459, 'Año': 2011},
  {'Horas': 82766, 'Año': 2010},
  {'Horas': 108771, 'Año': 2009},
  {'Horas': 8533, 'Año': 2008},
  {'Horas': 112794, 'Año': 2007},
  {'Horas': 90253, 'Año': 2006},
  {'Horas': 21339, 'Año': 2005},
  {'Horas': 127411, 'Año': 2004},
  {'Horas': 7744, 'Año': 2003},
  {'Horas': 238, 'Año': 2002},
  {'Horas': 250, 'Año': 2001},
  {'Horas': 70644, 'Año': 2000},
  {'Horas': 44, 'Año': 1999},
  {'Horas': 242, 'Año': 1997},
  {'Horas': 217, 'Año': 1995}]}

In [60]:
def UsersRecommend(anio):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == True) & (reviews['sentiment_analysis'] >0)].copy()
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    dfa['item_id'] = dfa['item_id'].astype(int)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
        if g in lid:
            top_games[i]=gnames[gnames['item_id'] == top_games[i]].head(1)['title'].iloc[0]
        else:
            top_games[i]=fnames[fnames['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [61]:
UsersRecommend(2013)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': "Garry's Mod",
 'Puesto 3': 'Dota 2'}

In [62]:
def UsersNotRecommend(anio):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == False) & (reviews['sentiment_analysis'] ==0)].copy()
    dfa['sentiment_analysis']=1
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa['item_id'] = dfa['item_id'].astype(int)
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
      if g in lid:
        top_games[i]=gnames[gnames['item_id'] == top_games[i]].head(1)['title'].iloc[0]
      else:
        top_games[i]=fnames[fnames['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    if len(top_games)<2:
      return{'Puesto 1':top_games[0],'Puesto 2':'','Puesto 3':''}
    elif len(top_games)<3:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':''}
    else:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [63]:
UsersNotRecommend(2013)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': 'Half-Life: Before',
 'Puesto 3': 'Call of Duty®: Ghosts'}

In [72]:
def sentiment_analysis(anio):
  dfs = pd.merge(reviews[['item_id','sentiment_analysis']],gnames[['item_id','release_date']],on='item_id',how='inner')
  dfs = dfs[dfs['release_date']==anio]
  dfs = dfs.groupby('sentiment_analysis').size().reset_index(name='count')
  dfs = dfs.sort_values('sentiment_analysis', ascending=True)
  dfs = dfs['count'].to_list()
  return{'Negative':dfs[0],'Neutral':dfs[1],'Positive':dfs[2]}

In [73]:
sentiment_analysis(2014)

{'Negative': 1040, 'Neutral': 1067, 'Positive': 4341}

In [74]:
def recomendacion_juego(idg):
    if (idg in lid) or (idg in lfid):
        gx=pd.DataFrame(games.loc[idg])
        gx=pd.pivot_table(gx,columns=['item_id', '2D', 'ACTION', 'ADVENTURE', 'ANIME', 'ARCADE',
       'ATMOSPHERIC', 'CASUAL', 'CLASSIC', 'CO-OP', 'COMEDY', 'CUTE',
       'DESIGN & ILLUSTRATION', 'DIFFICULT', 'EARLY ACCESS', 'EXPLORATION',
       'FPS', 'FAMILY FRIENDLY', 'FANTASY', 'FEMALE PROTAGONIST',
       'FIRST-PERSON', 'FREE TO PLAY', 'FUNNY', 'GORE', 'GREAT SOUNDTRACK',
       'HIDDEN OBJECT', 'HORROR', 'INDIE', 'LOCAL CO-OP', 'LOCAL MULTIPLAYER',
       'MASSIVELY MULTIPLAYER', 'MOVIE', 'MULTIPLAYER', 'ONLINE CO-OP',
       'OPEN WORLD', 'PIXEL GRAPHICS', 'PLATFORMER', 'POINT & CLICK', 'PUZZLE',
       'RPG', 'RPGMAKER', 'RACING', 'RETRO', 'ROGUE-LIKE', 'SANDBOX', 'SCI-FI',
       "SHOOT 'EM UP", 'SHOOTER', 'SIMULATION', 'SINGLEPLAYER', 'SPACE',
       'SPORTS', 'STORY RICH', 'STRATEGY', 'SURVIVAL', 'TACTICAL',
       'THIRD PERSON', 'TURN-BASED', 'UTILITIES', 'VIOLENT', 'VISUAL NOVEL',
       'ZOMBIES'])
        gx.drop(columns=['item_id'], inplace=True)
        xgames=games[games['item_id']!=idg].copy()
        xgames=xgames.reset_index()
        xgames.drop(columns=['index'],inplace=True)
        ygames=xgames.copy()
        xgames.drop(columns=['item_id'],inplace=True)
        matriz = cosine_similarity(gx,xgames)
        matrix = pd.DataFrame(matriz)
        matriz=matriz[0]
        reco=[]
        for i in range(5):
          imax=np.argmax(matriz)
          reco.append(imax)
          matriz[imax]=0
        top_games = []
        for i in reco:
          top_games.append(ygames.iloc[i]['item_id'])
        for i,g in enumerate(top_games):
          if g in lid:
            top_games[i]=gnames[gnames['item_id'] == top_games[i]]['title'].iloc[0]
          else:
            top_games[i]=fnames[fnames['item_id'] == top_games[i]]['item_name'].iloc[0]
        return {"juegos recomendados":
                 [{"Puesto 1":top_games[0]},
                  {"Puesto 2":top_games[1]},
                  {"Puesto 3":top_games[2]},
                  {"Puesto 4":top_games[3]},
                  {"Puesto 5":top_games[4]}]}
    else:
        return {"id de juego no se encuentra en la base de datos ":idg}

In [75]:
recomendacion_juego(440)

{'juegos recomendados': [{'Puesto 1': 'Big Money! Deluxe'},
  {'Puesto 2': 'Hammer Heads Deluxe'},
  {'Puesto 3': 'Iggle Pop Deluxe'},
  {'Puesto 4': 'Pizza Frenzy Deluxe'},
  {'Puesto 5': 'Typer Shark! Deluxe'}]}